# Quantum Classfier

In [ ]:
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.02

weights = initial_weights.copy()
train_loss_history = []
test_loss_history = []
acc_history = []

optimizer = AdamOptimizer(weights.shape, lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=epoch)

    epoch_loss = 0.0
    batches_count = 0

    for i in range(0, len(X_train), BATCH_SIZE):
        X_batch = X_train_shuffled[i:i + BATCH_SIZE]
        y_batch = y_train_shuffled[i:i + BATCH_SIZE]

        pred = qnn.forward(X_batch, weights)
        _, grads = qnn.backward(X_batch, weights)

        diff = pred - y_batch.reshape(-1, 1)
        loss = np.mean(diff ** 2)

        grad_modifier = 2 * diff

        batch_grads = np.mean(grad_modifier[:, :, np.newaxis] * grads, axis=0).squeeze()

        weights = optimizer.step(weights, batch_grads)

        epoch_loss += loss
        batches_count += 1

    test_outputs = qnn.forward(X_test, weights)

    test_diff = test_outputs - y_test.reshape(-1, 1)
    test_loss = np.mean(test_diff ** 2)

    predicted = (test_outputs > 0.5).astype(int).flatten()
    test_accuracy = np.mean(predicted == y_test.flatten())

    avg_loss = epoch_loss / batches_count
    train_loss_history.append(avg_loss)
    test_loss_history.append(test_loss)
    acc_history.append(test_accuracy)
    print(f"Epoch {epoch+1}/{EPOCHS} | Avg loss: {avg_loss:.4f} | Test Acc: {test_accuracy:.4f}")

test_outputs = qnn.forward(X_test, weights)
predicted = (test_outputs > 0.5).astype(int).flatten()
print(confusion_matrix(y_test, predicted))
print(f1_score(y_test, predicted))

In [ ]:
epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(14, 5))

# Plot 1: Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss_history, label='Train Loss', color='blue')
plt.plot(epochs, test_loss_history, label='Test Loss', color='red', linestyle='--')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot 2: Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, acc_history, label='Test Accuracy', color='green')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.show()